# Ligand Parameterization

Molecular mechanics forcefields need to be told how to treat each atom via a set of parameters. If there is a molecule (residue) in your system that your forcefield of choice does not already have parameters, you will need to build these yourself. In this example, we will see how QMzyme can automate the calculations required to parameterize a ligand in line with the RESP procedure (J. Phys. Chem. 1993, 97, 40, 10269–1028). 

In [8]:
import QMzyme
from QMzyme.data import PDB

### Initialize Model

In [2]:
# The initial pdb file should be preparred prior (hydrogens must be present).

pdb_file = PDB # here we are using package data. 
model = QMzyme.GenerateModel(PDB)


Charge information not present. QMzyme will try to guess region charges based on residue names consistent with AMBER naming conventions (i.e., aspartate: ASP --> Charge: -1, aspartic acid: ASH --> Charge: 0.). See QMzyme.data.residue_charges for the full set.

	Nonconventional Residues Found
	------------------------------
	EQU --> Charge: UNK, defaulting to 0

You can update charge information for nonconventional residues by running 
	>>>QMzyme.data.residue_charges.update({'3LETTER_RESNAME':INTEGER_CHARGE}). 
Note your changes will not be stored after you exit your session. It is recommended to only alter the residue_charges dictionary. If you alter the protein_residues dictionary instead that could cause unintended bugs in other modules (TruncationSchemes).



### Add Ligand Charge

In [3]:
QMzyme.data.residue_charges.update({'EQU': -1})

### Designate Ligand Region 

In [4]:
model.set_region(name='EQU', selection='resid 263 and resname EQU')
print("Ligand region: ", model.EQU)

Ligand region:  <QMzymeRegion EQU contains 37 atom(s) and 1 residue(s)>


### Build the QM Method

In [5]:
# For the purpose of this example we will forgo geometry optimization and only perform a single point energy 
# calculation and population analysis at the level used in the original RESP procedure (J. Phys. Chem. 1993, 97, 40, 10269–1028).

qm_method = QMzyme.QM_Method(
    basis_set='6-31G*', 
    functional='HF', 
    qm_input='SCF=Tight Pop=MK IOp(6/33=2,6/42=6,6/43=20)', 
    program='gaussian'
)

### Assign QM Method to Region

In [6]:
# Now we assign this method to our QMzymeRegion EQU
# We also need to specify the charge and multiplicity (mult) because QMzyme currently only guesses charges of standard amino acids.

qm_method.assign_to_region(region=model.EQU)

QMzymeRegion EQU has an estimated charge of -1.


### Write Calculation Input File

In [7]:
# Back to our Model

# QMzyme will know we only have one region with a calculation method set, 
# so it will logically create the input file for that scenario

model.write_input('EQU_resp')



File /Users/hrk/git/QMzyme/docs/Examples/QCALC/EQU_resp.com created.
